# Tax Revenues (Income!) in Barcelona

Open Data Barcelona provides lots of fun data about our city.

You can access it here: https://opendata-ajuntament.barcelona.cat

We will be examining average tax returns per neighborhood ("barri") in the years 2016 and 2015. Tax revenues are, naturally, a proxy for income, so we're really looking at home (taxable) income varies across the city.

The columns are in Catalan, so here's a quick explanation in English: 

Any = Year
Codi_Districte = District Code
Nom_Districte = District Name
Codi_Barri = Neighborhood Code
Nom_Barri = Neighborhood Name
Seccio_Censal = Cenus Tract Number
Import_Euros = Tax Revenue (average over all individuals in the census tract)

In [ ]:
# Let's begin by reading the file "2016_renda.csv"
# into a DataFrame:

import pandas as pd

df = pd.read_csv('2016_renda.csv')

In [30]:
#
# 1)
# Get the (5) barris with the highest average tax revenues
# (i.e. average over the census tracts in each barri)

df.groupby('Nom_Barri') \
  .mean() \
  .reset_index() \
  .sort_values('Import_Euros', ascending=False) \
  [:5] \
  [['Nom_Barri', 'Import_Euros']]

,Nom_Barri,Import_Euros
72,les Tres Torres,27626.818182
11,Pedralbes,26720.285714
17,Sant Gervasi - Galvany,24623.580645
18,Sant Gervasi - la Bonanova,23794.333333
23,Sarrià,23012.875000


In [31]:
#
# 2)
# Get the difference in mean revenue between the 
# poorest census tract and the richest, within 
# each district.
#
# You should return a DataFrame with 2 columns:
# The district name and the difference in reveneue.

def get_inequality(df):
    return df.Import_Euros.max() - df.Import_Euros.min()

df.groupby('Nom_Districte') \
  .apply(get_inequality) \
  .sort_values() \
  .reset_index(name='gap')

,Nom_Districte,gap
0,Ciutat Vella,8795
1,Gràcia,9303
2,Sant Andreu,9946
3,Nou Barris,10159
4,Sants-Montjuïc,11268
5,Horta-Guinardó,11657
6,Eixample,15176
7,Les Corts,16251
8,Sarrià-Sant Gervasi,16397
9,Sant Martí,19000


## Planning Your Attack

One pattern to make your code more legible, and to make it easier to break down big problems, is to ensure that your code can be read on two levels: one "declarative" level, where someone can read (or write) *what* will happen and another "imperative level", where someone can read (or write!) *how* the thing is happening.

Data preparation often involves a "pipeline", a uni-directional flow of transformations where the data is moved, one step at a time, towards the final format.

It's important, when you try to create a pipline, which can be a big problem, to make a plan.

One way to make a plan is to start from the final goal, and ask write out the following statement: 

1. "If I had ________ (INPUT), then it would be easy to make [FINAL GOAL], I would just need to ________ (step)."

Where you should think of INPUT as "data ______ in data structure ______".

That will be the final step of your pipeline. Now repeat the statement, with the FINAL GOAL being replaced with the INPUT of the previous step: 

2. "If I had ________ (INPUT), then it would be easy to make [PREVIOUS INPUT], I would just need to ________ (step)."

Let's see an example of this method of planning by working out an exercise:

In [32]:
#
# Your goal will be the following: 
#
# We want to understand the income variation 
# (or "spatial inequality") within each "barri".
# However, each barri is a different size.
# Larger barris will naturally have a greater
# variation, even if there isn't great variation
# between one block and the next, which is what
# we want to understand with spatial inequality.
# To deal with this, we will apply a naive solution
# of simply using the number of census tracts as
# a proxy for "physical size" of the barri. We 
# will then divide the income gap (difference between
# lowest and highest income tract) within each barri
# by the number of tracts as a way to "control for size".
# This will be our measure of "spatial inequality".
#
# Your job is to return a dataframe sorted by 
# spatial inequality, with any barri with one
# tract (0 inequality) removed.
#
#
# We will try to lay out a plan to solve the problem
# at hand with the process we just went over:

# 1. If I had a <<an extra column on the dataframe of 
#    the income gap divided by the number of tracts>>
#    then it would be easy to <<get the barris with 
#    highest and lowest normalized income gap>>, I 
#    would just need to <<sort the dataframe by that
#    column>>>.
#
# 2. If I had << A. a column for the income gap and 
#    B. a column for the number of tracts in a barri>>
#    then it would be easy to make << an extra column on the
#    dataframe of the income gap divided by the number of tracts>>
#    I would just need to <<divide one column by the other>>. 
#
#3b. If I had <<the raw data>>, then it would be easy to make
#    <<a column with the number of tracts>>, I would just need
#    to <<count the number of tracts per barri>>.
#
#3a. If I had <<the raw data>>, then it would be easy to make
#    <<a column with the income gap>>, I would just need to
#    <<calculate the income difference between tracts in each 
#    barri>>. 
#
# Now we can use this outline to write a declarative pipeline
# function (in the opposite order of the steps we wrote): 

def spatial_inequality_in_barcelona(df):
    df = add_income_diff_for_barris(df)
    df = add_num_tracts_per_barri(df)
    df = add_inequality(df)
    return inequality_by_barri(df)

# In the next exercises, you will write each of those functions,
# and in the end, use this function to compare barris based on
# their spatial inequality.

In [33]:
#
# 3)
# Write the function: "add_income_diff_for_barris"
#
# HINT: Make sure the returned dataframe is the
# same size as the original!
#

def add_diff(df):
    gap = get_inequality(df)
    return df.assign(gap=gap)

def add_income_diff_for_barris(df):
    return df.groupby('Nom_Barri') \
             .apply(add_diff) \
             .reset_index(drop=True)

df = add_income_diff_for_barris(df)
df

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros,gap
0,2016,9,Sant Andreu,58,Baró de Viver,6,8587,157
1,2016,9,Sant Andreu,58,Baró de Viver,7,8430,157
2,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,1,12507,6858
3,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,2,13475,6858
4,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,3,14066,6858
...,...,...,...,...,...,...,...,...
1063,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,26,29364,10380
1064,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,27,29364,10380
1065,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,28,29364,10380
1066,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,29,29364,10380


In [34]:
#
# 4)
# Create the function: "add_num_tracts_per_barri"

def add_num_tracts_per_barri(df):
    return df.groupby('Nom_Barri') \
             .apply(lambda df: df.assign(num_tracts = df.shape[0])) \
             .reset_index(drop=True)

df = add_num_tracts_per_barri(df)
df

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros,gap,num_tracts
0,2016,9,Sant Andreu,58,Baró de Viver,6,8587,157,2
1,2016,9,Sant Andreu,58,Baró de Viver,7,8430,157,2
2,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,1,12507,6858,25
3,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,2,13475,6858,25
4,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,3,14066,6858,25
...,...,...,...,...,...,...,...,...,...
1063,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,26,29364,10380,11
1064,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,27,29364,10380,11
1065,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,28,29364,10380,11
1066,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,29,29364,10380,11


In [35]:
#
# 5)
# Create the function: "add_inequality"

def add_inequality(df):
    return df.groupby('Nom_Barri') \
             .apply(lambda df: df.assign(inequality = df.gap/df.num_tracts)) \
             .reset_index(drop=True)

df = add_inequality(df)
df

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros,gap,num_tracts,inequality
0,2016,9,Sant Andreu,58,Baró de Viver,6,8587,157,2,78.500000
1,2016,9,Sant Andreu,58,Baró de Viver,7,8430,157,2,78.500000
2,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,1,12507,6858,25,274.320000
3,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,2,13475,6858,25,274.320000
4,2016,10,Sant Martí,64,Camp de l'Arpa del Clot,3,14066,6858,25,274.320000
...,...,...,...,...,...,...,...,...,...,...
1063,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,26,29364,10380,11,943.636364
1064,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,27,29364,10380,11,943.636364
1065,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,28,29364,10380,11,943.636364
1066,2016,5,Sarrià-Sant Gervasi,24,les Tres Torres,29,29364,10380,11,943.636364


In [ ]:
#
# 6)
# Add the function "inequality_by_barri"
# 
# Note that this function should probably 
# make sure that the dataframe has the
# same number of rows as number of barris
# (i.e. one barri per row).
#
# Also note that some barris have an inequality
# of 0, let's go ahead and remove them!

def inequality_by_barri(df):
    return df.drop_duplicates('Nom_Barri') \
             .drop(columns = ['Seccio_Censal']) \
             .sort_values('inequality') \
             .pipe(lambda df: df[df.inequality != 0])

inequality_by_barri(df)

In [43]:
# 
# 7) 
# Try out the function we wrote out in the planning
# phase, spatial_inequality_in_barcelona,
# does it work when given the raw data?
# 
# Now let's go ahead and "refactor"
# "Refactoring" means rewriting the code without
# changing the functionality. What we wrote works,
# and is great and legible. 
# 
# But maybe breaking it down into so many separate 
# steps, while didactic, could be considered overkill
# and maybe isn't the most efficient. You probably
# grouped by "Nom_Barri" at least 3 separate times!
#
# Try to rewrite the function spatial_inequality_in_barcelona
# to be more efficient (to only groupby Nom_Barri once!)
# and a bit shorter.


def add_inequality(df):
    gap = df.Import_Euros.max() - df.Import_Euros.min()
    sections = df.shape[0]
    return df.assign(gap=gap, 
                     sections=sections, 
                     inequality=gap/sections)

def spatial_inequality_in_barcelona(df):
    return df.groupby('Nom_Barri') \
             .apply(add_inequality) \
             .reset_index(drop=True) \
             .sort_values('inequality') \
             .pipe(lambda df: df[df.gap != 0]) \
             [['Nom_Barri', 'gap', 'sections', 'inequality']]  

spatial_inequality_in_barcelona(df)

,Nom_Barri,gap,sections,inequality
0,Baró de Viver,157,2,78.500000
1,Baró de Viver,157,2,78.500000
513,el Carmel,2545,22,115.681818
512,el Carmel,2545,22,115.681818
511,el Carmel,2545,22,115.681818
...,...,...,...,...
1009,la Vila Olímpica del Poblenou,8310,5,1662.000000
1010,la Vila Olímpica del Poblenou,8310,5,1662.000000
363,"Vallvidrera, el Tibidabo i les Planes",9995,3,3331.666667
362,"Vallvidrera, el Tibidabo i les Planes",9995,3,3331.666667


In [8]:
# Open Data Barcelona provides the tax data for years
# 2015 and 2016 in different csv's. Read in the tax data
# for year 2015 so we can see how incomes have changed
# between the years. 

#
# 8)
# Get the growth of the mean tax reveneue per census
# tract. Create a DataFrame that has the district, barri,
# and census tract as well as the difference in revenue
# between the years for each tract.
#
# Sort by the difference per tract.


def get_growth(df):
    growth = df.sort_values('Any').Import_Euros.diff().iloc[-1]
    df['growth'] = growth
    return df


both = pd.concat([df, pd.read_csv('2015_renda.csv')]).reset_index(drop=True)

both = both.groupby(['Nom_Barri', 'Seccio_Censal']) \
           .apply(get_growth) \
           .sort_values('growth')

both

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros,growth
405,2016,5,Sarrià-Sant Gervasi,23,Sarrià,5,14469,-9538.0
1473,2015,5,Sarrià-Sant Gervasi,23,Sarrià,5,24007,-9538.0
1574,2015,6,Gràcia,28,Vallcarca i els Penitents,9,26603,-4920.0
506,2016,6,Gràcia,28,Vallcarca i els Penitents,9,21683,-4920.0
433,2016,5,Sarrià-Sant Gervasi,25,Sant Gervasi - la Bonanova,33,23543,-4023.0
...,...,...,...,...,...,...,...,...
1576,2015,6,Gràcia,28,Vallcarca i els Penitents,11,17834,3845.0
404,2016,5,Sarrià-Sant Gervasi,23,Sarrià,4,29364,4341.0
1472,2015,5,Sarrià-Sant Gervasi,23,Sarrià,4,25023,4341.0
458,2016,5,Sarrià-Sant Gervasi,26,Sant Gervasi - Galvany,58,29364,4711.0


In [9]:
#
# 9)
# Get the mean growth per barri. 
# Sort by mean growth.


both.groupby('Nom_Barri').mean().sort_values('growth')

,Any,Codi_Districte,Codi_Barri,Seccio_Censal,Import_Euros,growth
Nom_Barri,,,,,,
la Vila Olímpica del Poblenou,2015.5,10.0,67.0,53.000000,21542.300000,-219.400000
Sant Genís dels Agudells,2015.5,7.0,39.0,91.000000,12655.800000,-174.800000
Ciutat Meridiana,2015.5,8.0,55.0,112.500000,7693.083333,-2.500000
Sants - Badal,2015.5,3.0,17.0,77.500000,13387.687500,6.375000
Vallcarca i els Penitents,2015.5,6.0,28.0,6.636364,18373.636364,24.363636
...,...,...,...,...,...,...
Pedralbes,2015.5,4.0,21.0,53.000000,26340.071429,760.428571
la Font d'en Fargues,2015.5,7.0,36.0,55.000000,17119.857143,761.428571
les Tres Torres,2015.5,5.0,24.0,25.000000,27199.954545,853.727273
